<center>
<table>
  <tr>
    <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h2><font color= "blue" size="+3">PyCon 2024 Tutorial</font></h2>
</center>

---

<center>
    <h3>Python Workflows to Extract and Plot Satellite Data Products along Tracks</h3>
    <h2><font color="red" size="+3">International Space Station - Exercise 2</font></h2>
</center>

## <font color="red"> Objectives</font>

- The International Space Station (ISS) makes 16 orbits of Earth, traveling through 16 sunrises and sunsets.
- ISS travels at a speed of five miles per second, circling Earth about every 90 minutes.
- We wrote a Python code that collects every 5 seconds (for over 5 hours) the position (latitude/longitude) and the date/time.
- For each ISS position, the code determines:
   - If it is a land or not (ocean)
   - The name of the country
   - The weather condition (temperature and wind speed)
   - The track id. A new track is set when ISS crosses the International Date Line (180th meridian)
- The collected time series data is written in a CSV file.

__The goal of this exercise is to read the CSV file using Pandas and then use GeoPandas and MovingPandas to perform data manipulation and visualization.__

To have more information on this exercise and to see what kind of manipulations we can do with the data, please check:

[Python Tool for Tracking the Movement of the International Space Station and Identifying the Weather Condition along the Track](https://github.com/astg606/pycon/tree/main/year2024/poster#readme)

## <font color="red">Import necessary modules</font>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import datetime as dt
from pathlib import Path

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
from shapely import geometry as shpgeom
from shapely import wkt as shpwkt

In [ ]:
import movingpandas as mpd

In [ ]:
import holoviews as hv
import hvplot.pandas 

## <font color="red">Read the CSV file</font>

In [ ]:
data_dir = "/Users/jkouatch/myTasks/PythonTraining/ASTG606/Materials/sat_data/ISS_Data/"
#data_dir = "/tljh-data/sat_data/ISS_Data"

In [ ]:
file_name = Path(data_dir) / "iss_timeseries_trajectories_20240303_104539.csv"

In [ ]:
df = pd.read_csv(file_name)

In [ ]:
df

## <font color="red">Perform Analyses</font>